In [3]:
import time
import random
import pandas as pd
import numpy as np
import requests
import json

user="2615aa2d9a9c4e1d63a9b961b018cc68"
pwd="q6x-RFUjwvc_rdhGyCLeKxkSrDiHNcA07e0pL6EO"
url_ravelry="https://api.ravelry.com/"

In [4]:
# get data, scrolling pages
def get_text_allpage(url,content='favorites'):
    output=requests.get(url,auth=requests.auth.HTTPBasicAuth(user, pwd),data={'page_size': 40}).json()
    num_pages=output['paginator']['last_page']
    data=output[content]
    for page in range(2,num_pages+1):
        try:
            output=requests.get(url,auth=requests.auth.HTTPBasicAuth(user, pwd),data={'page_size': 40,'page': page}).json()
        except:
            print('page loading error: <',url,'> page ',page)
            continue
        data.extend(output[content])
        #print(page)
        
    return data

def get_text_onepage(url):

    result = requests.get(url,auth=requests.auth.HTTPBasicAuth(user, pwd)).json()
    return result

## User ID scraping

In [ ]:
job_id=0
id_per_job=90#000
start=job_id*id_per_job
end=(job_id+1)*id_per_job
user_id_all=np.load('user_id_random.npy')
user_id=user_id_all[start:end]

user_df=[]
start_time=time.time()
for i in user_id:
    
    series={}
    #series['user_id']=i
    url="https://api.ravelry.com/people/"+str(i)+".json"
    #print(url)
    try:
        user_data=get_text_onepage(url)['user']
    except :
        continue
    user_name=user_data['username']
    location=user_data['location']
    series['user_name']=user_name
    series['location']=location
    user_df=pd.DataFrame(series,index=[i])
    user_df.index.name ="user_id"
    #user_df.set_index('user_id')
    filename='user_'+str(job_id)+'.csv'
    f = open(filename, 'a')
    user_df.to_csv(f,sep="^", header = False)
    f.close()
    
end_time=time.time()
print(end_time-start_time)


### user id subsetting

In [6]:
user_all=pd.read_csv("user_all.csv",header=0,sep='^') # all active users

In [32]:
user_all=user_all.set_index(['user_id'])
austin_name=['austin, texas','austin,texas',
             'austin,tx','austin, tx','austin']
nyc_name=['new york city,ny','new york city, ny','new york, ny',
          'new york,ny','new york,new york','new york ,new york',
          'new york city','new york']
user_nyc=user_all[user_all['location'].str.lower().isin(nyc_name)]
user_nyc.to_csv('user_nyc.csv',sep='^')
user_austin=user_all[user_all['location'].str.lower().isin(austin_name)]
user_austin.to_csv('user_austin.csv',sep='^')

In [37]:
df=user_all
df[df.location=='Towson']

,user_name,location
user_id,,
4261170,Tonk84,Towson
249066,alenec,Towson
1094762,Mysticalmudd,Towson
2826292,etquad,Towson
145286,robinsimons,Towson
2893105,LaverneEve,Towson
588901,Cutie01,Towson
62095,egadz,Towson
1605946,silverthim,Towson


## user favorite list to yarns

### single user

In [ ]:
def favorites_per_user(user_dat,filename):
    url_fav_list=url_ravelry+'people/'+user_dat.user_name+'/favorites/list.json'
    fav_list=get_text_allpage(url_fav_list,content='favorites')



    column_names = ["user_id", "username", "location",'time',"yarn_id",
              "yarn_permalink","yarn_weight"]
    df_fav=pd.DataFrame(columns = column_names)
    #df_fav=pd.DataFrame(current_user)

    for f in fav_list:
        current_fav={"user_id":user_dat.user_id, "username":user_dat.user_name,
              "location":user_dat.location}
        current_fav['time']=f['created_at'][:7]
        fav_type=f['type']

        if fav_type == 'pattern':
            content=f['favorited']
            item_id=content['id']
            item_url=url_ravelry+'patterns/'+str(item_id)+'.json'
            item_link=content['permalink']
            try:
                item=get_text_onepage(item_url)
            except:
                print("pattern loading error: ",item_link)
                file_object = open(filename+'_error', 'a')
                file_object.write(item_link+' \n')
                file_object.close()
                continue
            #print('ha')

            if item['pattern']['packs'] and \
                item['pattern']['packs'][0]['yarn_id']: 
                # if pattern specifies yarn
                #print('yarn')
                yarn_packs=item['pattern']['packs']
                for y in yarn_packs:
                    if y['yarn_id'] :
                        current_fav['yarn_id']=y['yarn_id']
                    if y['yarn']['permalink']:
                        current_fav['yarn_permalink']=y['yarn']['permalink']
                    if y['yarn_weight']:
                        current_fav['yarn_weight']=y['yarn_weight']['name']

                    #df_fav=df_fav.append(current_fav,ignore_index=True)  
                    index=str(user_dat.user_id)+' '+current_fav['time']
                    current_save=pd.DataFrame(current_fav,
                                             index=[index])
                    #add to file
                    f = open(filename, 'a')
                    current_save.to_csv(f,sep="^", header = False)
                    f.close()


### multiple users

In [ ]:
filename='user_fav.csv'
df_baltimore=df[df.location=='Baltimore'].iloc[:3]
for index, row in df_baltimore.iterrows():
    favorites_per_user(row,filename)
    print(row.user_id,row.user_name,'fav loaded')